In [ ]:
import pandas as pd
import numpy as np
import configparser
import os

pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)
## Using an ini file for credentials, otherwise providing defaults
uri = "bolt://localhost:7687"  # Hoặc thay đổi nếu cổng khác
username = "neo4j"  # Tên đăng nhập của Neo4j
password = "12345678"  # Mật khẩu của Neo4j
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(uri, auth=(username, password))
# Machine Learning Training & Evaluation
# Get and Prepare Data
df = gds.run_cypher('''
    MATCH(u:User)
    RETURN u.guid AS guid,
        u.wccId AS wccId,
        u.fraudRisk AS fraudRisk,
        u.fraudMoneyTransfer AS fraudMoneyTransfer,
        u.sharedIdsDegree AS sharedIdsDegree,
        u.p2pSharedCardPageRank AS p2pSharedCardPageRank,
        u.p2pSentPageRank AS p2pSentPageRank,
        u.p2pReceivedWeightedPageRank AS p2pReceivedWeightedPageRank,
        u.p2pReceivedWeightedDegree AS p2pReceivedWeightedDegree,
        u.ipDegree AS ipDegree,
        u.cardDegree AS cardDegree,
        u.deviceDegree AS deviceDegree,
        u.communitySize AS communitySize,
        u.partOfCommunity AS partOfCommunity
''')
df
# X: Tập dữ liệu đặc trưng đầu vào cho mô hình, loại bỏ các cột không cần thiết:
# Loại bỏ fraudRisk, fraudMoneyTransfer (vì đây là mục tiêu dự đoán).
# Loại bỏ wccId và guid (không mang giá trị cho mô hình học máy).
# X = df.drop(columns=['fraudRisk', 'fraudMoneyTransfer', 'wccId', 'guid', 'p2pSentPageRank', 'p2pReceivedWeightedPageRank','cardDegree','p2pReceivedWeightedDegree', 'p2pSharedCardPageRank', 'deviceDegree'])
X = df.drop(columns=['fraudRisk', 'fraudMoneyTransfer', 'wccId', 'guid'])
# y: Biến mục tiêu (target variable):
# df.fraudRisk - df.fraudMoneyTransfer:
# Nếu fraudRisk = 1 và fraudMoneyTransfer = 0 → tài khoản nguy cơ rủi ro → y = 1.
# Nếu fraudRisk = 1 và fraudMoneyTransfer = 1 → tài khoản đã chuyển tiền gian lận → y = 0.
# Nếu fraudRisk = 0 → tài khoản an toàn → y = 0.
y = df.fraudRisk - df.fraudMoneyTransfer

In [1]:
import os
import pickle
from sklearn.model_selection import train_test_split

# Tên file lưu
split_file = "splitted_data.pkl"

if os.path.exists(split_file):
    print("Tải data đã lưu")
    with open(split_file, "rb") as f:
        data_splits = pickle.load(f)
    # Lấy các tập dữ liệu từ dictionary
    X_train = data_splits["X_train"]
    X_val = data_splits["X_val"]
    X_test = data_splits["X_test"]
    y_train = data_splits["y_train"]
    y_val = data_splits["y_val"]
    y_test = data_splits["y_test"]
    X_train_val = data_splits["X_train_val"]
    y_train_val = data_splits["y_train_val"]
else:
    print("Lưu data vào file")
    # Giả sử X và y đã được định nghĩa sẵn
    # Chia dữ liệu thành tập train+val (80%) và test (20%)
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Chia tiếp tập train_val thành train (75% của train_val) và val (25% của train_val)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
    )

    # Lưu lại toàn bộ các tập dưới dạng dictionary, bao gồm cả train_val
    data_splits = {
        "X_train": X_train,
        "X_val": X_val,
        "X_test": X_test,
        "y_train": y_train,
        "y_val": y_val,
        "y_test": y_test,
        "X_train_val": X_train_val,
        "y_train_val": y_train_val
    }
    
    with open(split_file, "wb") as f:
        pickle.dump(data_splits, f)
    print("Đã lưu vào file: ", split_file)


Tải data đã lưu


In [2]:
import numpy as np  
print("Hình dạng của X_train:", np.array(X_train).shape)
print("Hình dạng của y_train:", np.array(y_train).shape)


Hình dạng của X_train: (20238, 10)
Hình dạng của y_train: (20238,)


In [3]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# Định nghĩa cross-validation phù hợp đảm bảo tỷ lệ fraud/non-fraud được giữ nguyên trong mỗi fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [25]:
from sklearn.tree import DecisionTreeClassifier

# Tạo Pipeline gồm: chuẩn hóa → SMOTE → Decision Tree
pipeline_dt = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
])

# Grid các siêu tham số (thêm class_weight)
dt_param_grid = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [None, 5, 10, 20, 30],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4],
    'dt__max_features': ['sqrt', 'log2', None],
    'dt__class_weight': [None,'balanced']
}

# Sử dụng GridSearchCV với scoring = 'average_precision'
dt_grid = GridSearchCV(
    estimator=pipeline_dt,
    param_grid=dt_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# Huấn luyện mô hình trên tập train/val
dt_grid.fit(X_train_val, y_train_val)

# Lấy mô hình tốt nhất
best_model_dt = dt_grid.best_estimator_
print("Best Parameters:", dt_grid.best_params_)

# Dự đoán trên tập test
y_test_pred = best_model_dt.predict(X_test)

# In kết quả đánh giá
print("\nKết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))



Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best Parameters: {'dt__class_weight': None, 'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': None, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 5}

Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      6705
           1       0.05      0.48      0.10        42

    accuracy                           0.94      6747
   macro avg       0.53      0.71      0.53      6747
weighted avg       0.99      0.94      0.97      6747



In [14]:
from sklearn.ensemble import RandomForestClassifier

# Tạo Pipeline gồm: chuẩn hóa → SMOTE → Random Forest
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42))
])

# Grid siêu tham số (thêm class_weight vào)
rf_param_grid = {
    'rf__n_estimators': [200, 500],
    'rf__max_depth': [5, 10],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__max_features': ['sqrt'],
    'rf__bootstrap': [True],
    'rf__class_weight': ['balanced']
}

# GridSearchCV với PR AUC
rf_grid = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=rf_param_grid,
    cv=cv,
    scoring = 'average_precision',
    n_jobs=-1,
    verbose=1
)

# Huấn luyện trên tập train/val
rf_grid.fit(X_train_val, y_train_val)

# Lấy mô hình tốt nhất
best_model_rf = rf_grid.best_estimator_
print("Random Forest - Best Parameters:", rf_grid.best_params_)

# Dự đoán trên tập test
y_test_pred = best_model_rf.predict(X_test)

# Đánh giá
print("\nRandom Forest - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Random Forest - Best Parameters: {'rf__bootstrap': True, 'rf__class_weight': 'balanced', 'rf__max_depth': 10, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}

Random Forest - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      6705
           1       0.07      0.57      0.12        42

    accuracy                           0.95      6747
   macro avg       0.53      0.76      0.55      6747
weighted avg       0.99      0.95      0.97      6747



In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# 1. Tạo pipeline gồm: chuẩn hóa → SMOTE → SVM
pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
])

# 2. Không gian siêu tham số: thêm class_weight
svm_random_grid = {
    'svc__C': uniform(0.1, 10),
    'svc__gamma': ['scale'],
    'svc__kernel': ['rbf'],
    'svc__class_weight': ['balanced']
}

# 3. Thiết lập RandomizedSearchCV với scoring = average_precision
svm_search = RandomizedSearchCV(
    estimator=pipeline_svm,
    param_distributions=svm_random_grid,
    n_iter=5,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# 4. Huấn luyện mô hình
svm_search.fit(X_train_val, y_train_val)

# 5. Mô hình tốt nhất
best_model_svm = svm_search.best_estimator_
print("SVM - Best Parameters:", svm_search.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_svm.predict(X_test)

# 7. Đánh giá mô hình
print("\nSVM - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 5 candidates, totalling 25 fits
SVM - Best Parameters: {'svc__C': 1.6601864044243653, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}

SVM - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.82      0.90      6705
           1       0.03      0.90      0.06        42

    accuracy                           0.82      6747
   macro avg       0.52      0.86      0.48      6747
weighted avg       0.99      0.82      0.90      6747



In [31]:
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# 1. Tạo pipeline gồm: chuẩn hóa → SMOTE → GaussianNB
pipeline_nb = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('nb', GaussianNB())
])
# Danh sách các priors cần thử nghiệm
priors_grid = [
    [0.99, 0.01],
    [0.95, 0.05],
    [0.9, 0.1],
    [0.85, 0.15],
    [0.8, 0.2]
]

# Grid param với priors là siêu tham số
nb_param_grid = {
    'nb__var_smoothing': [1e-9, 1e-8],
    'nb__priors': priors_grid
}

# 3. Thiết lập GridSearchCV với scoring = 'average_precision'
nb_grid = GridSearchCV(
    estimator=pipeline_nb,
    param_grid=nb_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện mô hình trên tập train/val
nb_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_nb = nb_grid.best_estimator_
print("Naïve Bayes - Best Parameters:", nb_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_nb.predict(X_test)

# 7. Đánh giá mô hình
print("\nNaïve Bayes - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Naïve Bayes - Best Parameters: {'nb__priors': [0.99, 0.01], 'nb__var_smoothing': 1e-08}

Naïve Bayes - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      6705
           1       0.03      0.52      0.06        42

    accuracy                           0.89      6747
   macro avg       0.51      0.71      0.50      6747
weighted avg       0.99      0.89      0.94      6747



In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 1. Tạo pipeline: chuẩn hóa → SMOTE → Bagging(DecisionTree)
pipeline_bag = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('bag', BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=42),
        random_state=42))
])

# 2. Không gian siêu tham số cho Bagging + class_weight
bagging_param_grid = {
    'bag__n_estimators': [10, 50, 100],
    'bag__max_samples': [0.5, 1.0],
    'bag__max_features': [0.5, 1.0],
    'bag__bootstrap': [True, False],
    'bag__estimator__class_weight': ['balanced']
}

# 3. GridSearchCV với scoring = 'average_precision'
bagging_grid = GridSearchCV(
    estimator=pipeline_bag,
    param_grid=bagging_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện trên tập train/val
bagging_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_bag = bagging_grid.best_estimator_
print("Bagging - Best Parameters:", bagging_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_bag.predict(X_test)

# 7. Đánh giá mô hình
print("\nBagging - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Bagging - Best Parameters: {'bag__bootstrap': False, 'bag__estimator__class_weight': 'balanced', 'bag__max_features': 0.5, 'bag__max_samples': 1.0, 'bag__n_estimators': 100}

Bagging - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6705
           1       0.80      0.10      0.17        42

    accuracy                           0.99      6747
   macro avg       0.90      0.55      0.58      6747
weighted avg       0.99      0.99      0.99      6747



In [6]:
from xgboost import XGBClassifier

# 1. Tạo pipeline: chuẩn hóa → SMOTE → XGBoost
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(eval_metric='logloss', random_state=42))
])

# 2. Định nghĩa không gian siêu tham số (có thêm scale_pos_weight)
xgb_param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__scale_pos_weight': [1, 3, 5, 10]  
}

# 3. Thiết lập GridSearchCV
xgb_grid = GridSearchCV(
    estimator=pipeline_xgb,
    param_grid=xgb_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện mô hình trên tập train/val
xgb_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_xgb = xgb_grid.best_estimator_
print("XGBoost - Best Parameters:", xgb_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_xgb.predict(X_test)

# 7. Đánh giá kết quả
print("\nXGBoost - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 192 candidates, totalling 960 fits
XGBoost - Best Parameters: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 200, 'xgb__scale_pos_weight': 3, 'xgb__subsample': 0.8}

XGBoost - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      6705
           1       0.06      0.52      0.11        42

    accuracy                           0.95      6747
   macro avg       0.53      0.74      0.54      6747
weighted avg       0.99      0.95      0.97      6747



In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Tạo Pipeline: chuẩn hóa → Decision Tree (không dùng SMOTE)
pipeline_dt = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=42))
])

# Grid các siêu tham số, chỉ sử dụng class_weight thay cho SMOTE
dt_param_grid = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [None, 5, 10, 20, 30],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4],
    'dt__max_features': ['sqrt', 'log2', None],
    'dt__class_weight': ['balanced'] 
}

# Thiết lập GridSearchCV
dt_grid = GridSearchCV(
    estimator=pipeline_dt,
    param_grid=dt_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# Huấn luyện mô hình
dt_grid.fit(X_train_val, y_train_val)

# Lấy mô hình tốt nhất
best_model_dt = dt_grid.best_estimator_
print("Best Parameters:", dt_grid.best_params_)

# Dự đoán trên tập test
y_test_pred = best_model_dt.predict(X_test)

# Đánh giá kết quả
print("\nKết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Parameters: {'dt__class_weight': 'balanced', 'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': None, 'dt__min_samples_leaf': 4, 'dt__min_samples_split': 10}

Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96      6705
           1       0.04      0.48      0.08        42

    accuracy                           0.93      6747
   macro avg       0.52      0.70      0.52      6747
weighted avg       0.99      0.93      0.96      6747



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline  # dùng sklearn vì không còn SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Tạo Pipeline gồm: chuẩn hóa → Random Forest (không dùng SMOTE)
pipeline_rf = Pipeline([
    ('rf', RandomForestClassifier(random_state=42))
])

# Grid siêu tham số với class_weight
rf_param_grid = {
    'rf__n_estimators': [200, 500],
    'rf__max_depth': [5, 10],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__max_features': ['sqrt'],
    'rf__bootstrap': [True],
    'rf__class_weight': ['balanced']
}

# GridSearchCV với scoring = average_precision (PR AUC)
rf_grid = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=rf_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# Huấn luyện trên tập train/val
rf_grid.fit(X_train_val, y_train_val)

# Lấy mô hình tốt nhất
best_model_rf = rf_grid.best_estimator_
print("Random Forest - Best Parameters:", rf_grid.best_params_)

# Dự đoán trên tập test
y_test_pred = best_model_rf.predict(X_test)

# Đánh giá
print("\nRandom Forest - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Random Forest - Best Parameters: {'rf__bootstrap': True, 'rf__class_weight': 'balanced', 'rf__max_depth': 10, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 500}

Random Forest - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      6705
           1       0.12      0.40      0.19        42

    accuracy                           0.98      6747
   macro avg       0.56      0.69      0.59      6747
weighted avg       0.99      0.98      0.98      6747



In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# 1. Tạo pipeline gồm: chuẩn hóa → SMOTE → SVM
pipeline_svm = Pipeline([
    ('svc', SVC(probability=True, random_state=42))
])

# 2. Không gian siêu tham số: thêm class_weight
svm_random_grid = {
    'svc__C': uniform(0.1, 10),
    'svc__gamma': ['scale'],
    'svc__kernel': ['rbf'],
    'svc__class_weight': ['balanced']
}

# 3. Thiết lập RandomizedSearchCV với scoring = average_precision
svm_search = RandomizedSearchCV(
    estimator=pipeline_svm,
    param_distributions=svm_random_grid,
    n_iter=5,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# 4. Huấn luyện mô hình
svm_search.fit(X_train_val, y_train_val)

# 5. Mô hình tốt nhất
best_model_svm = svm_search.best_estimator_
print("SVM - Best Parameters:", svm_search.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_svm.predict(X_test)

# 7. Đánh giá mô hình
print("\nSVM - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 5 candidates, totalling 25 fits
SVM - Best Parameters: {'svc__C': 9.60714306409916, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}

SVM - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.90      0.94      6705
           1       0.02      0.33      0.04        42

    accuracy                           0.89      6747
   macro avg       0.51      0.61      0.49      6747
weighted avg       0.99      0.89      0.94      6747



In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Tạo pipeline
pipeline_nb = Pipeline([
    ('scaler', StandardScaler()),
    ('nb', GaussianNB())
])

# Danh sách các priors cần thử nghiệm
priors_grid = [
    [0.99, 0.01],
    [0.95, 0.05],
    [0.9, 0.1],
    [0.85, 0.15],
    [0.8, 0.2]
]

# Grid param với priors là siêu tham số
nb_param_grid = {
    'nb__var_smoothing': [1e-9, 1e-8],
    'nb__priors': priors_grid
}

# GridSearchCV
nb_grid = GridSearchCV(
    estimator=pipeline_nb,
    param_grid=nb_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện mô hình
nb_grid.fit(X_train_val, y_train_val)

# 5. Mô hình tốt nhất
best_model_nb = nb_grid.best_estimator_
print("Naïve Bayes - Best Parameters:", nb_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_nb.predict(X_test)

# 7. Đánh giá kết quả
print("\nNaïve Bayes - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Naïve Bayes - Best Parameters: {'nb__priors': [0.99, 0.01], 'nb__var_smoothing': 1e-08}

Naïve Bayes - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      6705
           1       0.03      0.48      0.05        42

    accuracy                           0.89      6747
   macro avg       0.51      0.68      0.49      6747
weighted avg       0.99      0.89      0.93      6747



In [23]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 1. Tạo pipeline: chuẩn hóa → SMOTE → Bagging(DecisionTree)
pipeline_bag = Pipeline([
    ('bag', BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=42),
        random_state=42))
])

# 2. Không gian siêu tham số cho Bagging + class_weight
bagging_param_grid = {
    'bag__n_estimators': [10, 50, 100],
    'bag__max_samples': [0.5, 1.0],
    'bag__max_features': [0.5, 1.0],
    'bag__bootstrap': [True, False],
    'bag__estimator__class_weight': ['balanced']
}

# 3. GridSearchCV với scoring = 'average_precision'
bagging_grid = GridSearchCV(
    estimator=pipeline_bag,
    param_grid=bagging_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện trên tập train/val
bagging_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_bag = bagging_grid.best_estimator_
print("Bagging - Best Parameters:", bagging_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_bag.predict(X_test)

# 7. Đánh giá mô hình
print("\nBagging - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Bagging - Best Parameters: {'bag__bootstrap': True, 'bag__estimator__class_weight': 'balanced', 'bag__max_features': 1.0, 'bag__max_samples': 1.0, 'bag__n_estimators': 100}

Bagging - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6705
           1       0.60      0.14      0.23        42

    accuracy                           0.99      6747
   macro avg       0.80      0.57      0.61      6747
weighted avg       0.99      0.99      0.99      6747



In [26]:
from xgboost import XGBClassifier

# 1. Tạo pipeline: chuẩn hóa → SMOTE → XGBoost
pipeline_xgb = Pipeline([
    ('xgb', XGBClassifier(eval_metric='logloss', random_state=42))
])

# 2. Định nghĩa không gian siêu tham số
xgb_param_grid = {
    'xgb__n_estimators': [200, 500],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__scale_pos_weight': [1, 3, 5, 10]  
}

# 3. Thiết lập GridSearchCV
xgb_grid = GridSearchCV(
    estimator=pipeline_xgb,
    param_grid=xgb_param_grid,
    cv=cv,  
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện mô hình trên tập train/val
xgb_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_xgb = xgb_grid.best_estimator_
print("XGBoost - Best Parameters:", xgb_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_xgb.predict(X_test)

# 7. Đánh giá kết quả
print("\nXGBoost - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Fitting 5 folds for each of 192 candidates, totalling 960 fits
XGBoost - Best Parameters: {'xgb__colsample_bytree': 0.8, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 5, 'xgb__n_estimators': 500, 'xgb__scale_pos_weight': 10, 'xgb__subsample': 1.0}

XGBoost - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      6705
           1       0.26      0.29      0.27        42

    accuracy                           0.99      6747
   macro avg       0.63      0.64      0.63      6747
weighted avg       0.99      0.99      0.99      6747



In [29]:
from sklearn.neighbors import KNeighborsClassifier

# 1. Tạo pipeline gồm: chuẩn hóa → SMOTE → KNN
pipeline_knn = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 2. Định nghĩa không gian siêu tham số
knn_param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]  # 1: Manhattan, 2: Euclidean
}

# 3. Sử dụng GridSearchCV với scoring = 'average_precision' (PR AUC)
knn_grid = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=knn_param_grid,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# 4. Huấn luyện mô hình trên tập train/validation
knn_grid.fit(X_train_val, y_train_val)

# 5. Lấy mô hình tốt nhất
best_model_knn = knn_grid.best_estimator_
print("KNN - Best Parameters:", knn_grid.best_params_)

# 6. Dự đoán trên tập test
y_test_pred = best_model_knn.predict(X_test)

# 7. Đánh giá trên tập test
print("\nKNN - Kết quả trên tập Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
KNN - Best Parameters: {'knn__n_neighbors': 9, 'knn__p': 1, 'knn__weights': 'distance'}

KNN - Kết quả trên tập Test:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6705
           1       0.00      0.00      0.00        42

    accuracy                           0.99      6747
   macro avg       0.50      0.50      0.50      6747
weighted avg       0.99      0.99      0.99      6747

